In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

<Axes: xlabel='count', ylabel='domain'>

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<Axes: xlabel='count', ylabel='posted_by'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

/tmp/ipykernel_419477/2178239382.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby(["domain", "posted_by"])


domain         \
                                            count unique   
domain               posted_by                             
americanthinker.com  MtnClimber               131      1   
                     Red Badger                 1      1   
                     SeekAndFind               31      1   
                     george76                   8      1   
apnews.com           Red Badger                 2      1   
                     SeekAndFind                2      1   
                     E. Pluribus Unum          45      1   
babylonbee.com       E. Pluribus Unum           1      1   
bbc.com              Red Badger                 2      1   
                     SeekAndFind                2      1   
                     george76                   2      1   
                     marcusmaximus              4      1   
                     nickcarraway               5      1   
breitbart.com        MtnClimber                 1      1   
                     Red Badger                 8      1   
                     george76                   4      1   
                     ChicagoConservative27    308      1   
                     Macho MAGA Man            12      1   
                     Morgana                   12      1   
cnbc.com             Red Badger                11      1   
                     SeekAndFind                6      1   
                     E. Pluribus Unum           2      1   
                     marcusmaximus              3      1   
dailymail.co.uk      Red Badger                11      1   
                     SeekAndFind                1      1   
                     george76                   1      1   
                     marcusmaximus              7      1   
                     nickcarraway               5      1   
                     ChicagoConservative27      1      1   
                     Macho MAGA Man             1      1   
                     Morgana                   34      1   
foxnews.com          MtnClimber                 2      1   
                     SeekAndFind                3      1   
                     george76                  10      1   
                     E. Pluribus Unum           2      1   
                     marcusmaximus              8      1   
                     nickcarraway               3      1   
                     ChicagoConservative27      6      1   
                     Macho MAGA Man             1      1   
                     Morgana                    1      1   
justthenews.com      MtnClimber                 1      1   
                     Red Badger                 8      1   
                     SeekAndFind                1      1   
                     george76                   5      1   
                     E. Pluribus Unum           4      1   
msn.com              Red Badger                 1      1   
                     SeekAndFind               19      1   
                     E. Pluribus Unum           1      1   
                     marcusmaximus             13      1   
                     nickcarraway               2      1   
                     ChicagoConservative27      2      1   
newsmax.com          marcusmaximus              1      1   
                     ChicagoConservative27      2      1   
nypost.com           MtnClimber                 1      1   
                     Red Badger                 5      1   
                     SeekAndFind                3      1   
                     george76                  22      1   
                     E. Pluribus Unum           3      1   
                     marcusmaximus             10      1   
                     nickcarraway              44      1   
                     ChicagoConservative27     92      1   
                     Macho MAGA Man             1      1   
                     Morgana                    3      1   
nytimes.com          E. Pluribus Unum          58      1   
                     ni

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

/tmp/ipykernel_419477/1921096508.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby('title').


,timestamp,replies
title,,
Leo 14th,2025-05-08 18:37:18,201.0
When Did Rock & Roll Die? A Statistical Analysis,2025-05-05 10:49:23,163.0
